In [ ]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 26.3 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# print(df['symbol'])

# actual_tickers = list(set(df['symbol'].to_list()))
# len(actual_tickers)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from torch_geometric.nn import Node2Vec
import torch
import matplotlib.pyplot as plt
import yfinance as yf
import pandas as pd
from functools import wraps
import numpy as np
import csv

ModuleNotFoundError: No module named 'torch_geometric'

In [ ]:
import pandas as pd

## Data Retrieval

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

In [ ]:
import pandas as pd

### The dataset

In [ ]:

class DataRetrieval:
    """Downloads stocks data an aggregates with technical stock statistics and covariance from other stocks
    adds train/test split
    """

    def __init__(self):
        pass

    """
    Param
    ----------
        start_date : str
            start date of the data
        end_date : str
            end date of the data
        ticker_list : list
            a list of stock tickers
        time_interval : str
            download frequency: 1m, 2m, 5m, 15m, 30m, 60m, 90m, 1h, 1d, 5d, 1wk, 1mo, 3mo
    """


    @staticmethod
    def clean_table(hist_data) -> pd.DataFrame:
        hist_data.index = pd.to_datetime(hist_data.index, format='%Y-%m-%d') # Add format string
        hist_data.index = hist_data.index.strftime('%Y-%m-%d')
        # Reset index
        hist_data = hist_data.reset_index()
        # Add day 0-n
        hist_data["day"] = range(len(hist_data))
        # Lowercase all titles
        hist_data.columns = hist_data.columns.str.lower()

        return hist_data

    @staticmethod
    def add_covariance(hist_data, lookback) -> pd.DataFrame:
        # add covariance matrix as states
        hist_data=hist_data.sort_values(['date','tic'],ignore_index=True)
        hist_data.index = hist_data.date.factorize()[0]

        cov_list = []
        return_list = []

        # look back is one year
        for i in range(lookback,len(hist_data.index.unique())):
            data_lookback = hist_data.loc[i-lookback:i,:]
            price_lookback = data_lookback.pivot_table(index = 'date',columns = 'tic', values = 'close')
            return_lookback = price_lookback.pct_change().dropna()
            return_list.append(return_lookback)

            covs = return_lookback.cov().values
            cov_list.append(covs)

        df_cov = pd.DataFrame({'date':hist_data.date.unique()[lookback:],'cov_list':cov_list,'return_list':return_list})
        hist_data = hist_data.merge(df_cov, on='date')
        hist_data = hist_data.sort_values(['date','tic']).reset_index(drop=True)
        return hist_data


    def download_data(self, ticker_list, start_date, end_date, time_interval, tech_indicator_list):
        self.start = start_date
        self.end = end_date
        self.time_interval = time_interval

        # Download and save the data in a pandas DataFrame
        start_date = pd.Timestamp(start_date)
        end_date = pd.Timestamp(end_date)
        data_df = pd.DataFrame()

        # ticker_list.append("VIXY") # append VIXY as reference

        for tic in ticker_list:
            ticker = yf.Ticker(tic)
            ticker_historical = ticker.history(start=start_date, end=end_date, interval=self.time_interval)
            # print(tic, len(ticker_historical))

            # Clean data
            ticker_historical_cleaned = self.clean_table(ticker_historical)
            # Add ticker
            ticker_historical_cleaned["tic"] = tic

            data_df = pd.concat([data_df, ticker_historical_cleaned])

        # Sort by day
        data_df = data_df.sort_values(by=['day'])

        return data_df


In [ ]:
#ticker_list = [ticker for ticker in relation_data]

TECH_INDICATOR_LIST     = ['middle', 'rsi']

In [ ]:
!pip install openpyxl
import pandas as pd

file_path = '/content/drive/MyDrive/tickers.xlsx'
df = pd.read_excel(file_path, engine='openpyxl')

print(df.columns)
df = df.drop(df.columns[0], axis=1)
#print(ticker_list)

Index(['Date', 'A', 'AAL', 'AAP', 'AAPL', 'ABBV', 'ABC', 'ABT', 'ACGL', 'ACN',
       ...
       'WYNN', 'XEL', 'XOM', 'XRAY', 'XYL', 'YUM', 'ZBH', 'ZBRA', 'ZION',
       'ZTS'],
      dtype='object', length=502)


In [ ]:
import pandas as pd



df_embed = pd.read_csv('/content/drive/MyDrive/full_df.csv')


In [ ]:
#ticker_list = df.columns

In [ ]:
print(df_embed['symbol'])

actual_tickers = list(set(df_embed['symbol'].to_list()))
len(actual_tickers)

0       REG
1      KEYS
2       AEP
3       MRO
4      ALLE
       ... 
317    SEDG
318      CI
319      CE
320    FOXA
321    GOOG
Name: symbol, Length: 322, dtype: object


322

In [ ]:
#df.to_csv('/content/drive/MyDrive/node2Vec_sampledata.csv', index=False)

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/node2Vec_sampledata.csv')

In [ ]:
print(df)
df['close_pct_change'] = (df['close'] - df['close'].shift(1)) / df['close'].shift(1)

              date        open        high         low       close  \
0       2016-01-04   45.110001   45.110001   44.000000   44.919998   
1       2016-01-04  135.361578  140.104897  133.783488  138.073349   
2       2016-01-04   13.366961   13.472861   13.335583   13.472861   
3       2016-01-04    9.488014    9.742231    9.217396    9.455212   
4       2016-01-04   60.795555   61.092220   59.866669   60.821110   
...            ...         ...         ...         ...         ...   
646302  2024-03-28  299.533755  302.329391  299.164335  300.711914   
646303  2024-03-28    7.845783    7.984472    7.806158    7.954752   
646304  2024-03-28  207.182272  207.272105  202.341107  205.854691   
646305  2024-03-28  685.529428  686.535341  677.700793  684.284424   
646306  2024-03-28  611.028621  614.314344  608.571781  613.145874   

            volume  dividends  stock splits   day   tic  adj close  
0         831700.0        0.0           0.0     0  SNPS        NaN  
1        1935300.0   

In [ ]:
output_file_path = '/content/drive/MyDrive/test_data_320_SP500.xlsx'
df.to_excel(output_file_path, index=False)

In [ ]:
file_path = '/content/drive/MyDrive/test_data_320_SP500.xlsx'
df = pd.read_excel(file_path)

In [ ]:
quantile1 = pd.read_csv('/content/drive/MyDrive/saved_q01_.csv', index_col=0)

In [ ]:
print(quantile1)

             A       AAL      AAPL      ABBV       ABT      ACGL       ACN  \
A     1.000000  0.430264  0.455135  0.451925  0.596241  0.353478  0.596027   
AAL   0.430264  1.000000  0.307744  0.280196  0.406048  0.264996  0.368987   
AAPL  0.455135  0.307744  1.000000  0.269007  0.418096  0.249338  0.456026   
ABBV  0.451925  0.280196  0.269007  1.000000  0.479327  0.142457  0.266560   
ABT   0.596241  0.406048  0.418096  0.479327  1.000000  0.400294  0.541320   
...        ...       ...       ...       ...       ...       ...       ...   
XOM   0.465948  0.252601  0.324733  0.285589  0.484625  0.291843  0.409552   
XYL   0.564414  0.387567  0.374624  0.327075  0.462696  0.374706  0.502788   
YUM   0.439659  0.350732  0.344477  0.237796  0.469300  0.331982  0.465366   
ZBRA  0.449857  0.349323  0.353535  0.241263  0.380962  0.183166  0.373879   
ZTS   0.478929  0.321288  0.342448  0.374725  0.522309  0.278289  0.457981   

          ADBE       ADI       ADM  ...       WMT       WRB    

In [ ]:
import pandas as pd
import torch
from torch_geometric.data import Data

# Load quantile correlation matrices
quantile1 = pd.read_csv('/content/drive/MyDrive/saved_q01_.csv', index_col=0)
quantile2 = pd.read_csv('/content/drive/MyDrive/saved_q02_.csv', index_col=0)
quantile3 = pd.read_csv('/content/drive/MyDrive/saved_q03_.csv', index_col=0)

final_stocks = pd.read_csv('/content/drive/MyDrive/final_columns.csv')
list_stocks = final_stocks['stocks'].to_list()

common_stocks = list(set(list_stocks).intersection(quantile1.index))

# Filter quantile DataFrames based on common stocks
quantile1 = quantile1.loc[common_stocks, common_stocks]
quantile2 = quantile2.loc[common_stocks, common_stocks]
quantile3 = quantile3.loc[common_stocks, common_stocks]

# Ensure the matrices have the same shape and stocks
assert quantile1.shape == quantile2.shape == quantile3.shape
stocks = quantile1.index.tolist()

# Ensure the matrices have the same shape and stocks
assert quantile1.shape == quantile2.shape == quantile3.shape
stocks = quantile1.index.tolist()

edge_index_list = []
edge_features_list = []

# Enumerate stocks to get their indices
enumerated_stocks = {stock: idx for idx, stock in enumerate(stocks)}

# Iterate over the upper triangle of the correlation matrices to avoid duplicate edges
for i, stock1 in enumerate(stocks):
    for j, stock2 in enumerate(stocks):
        if i != j:
            edge_index_list.append([enumerated_stocks[stock1], enumerated_stocks[stock2]])
            edge_features_list.append([
                quantile1.iloc[i, j],
                quantile2.iloc[i, j],
                quantile3.iloc[i, j]
            ])

edge_index = torch.tensor(edge_index_list, dtype=torch.long).t().contiguous()
edge_features = torch.tensor(edge_features_list, dtype=torch.float)

# Create dummy node features (replace with actual features if available)
num_stocks = len(stocks)
node_features = torch.randn(num_stocks, 5)  # Example: 5 random features per stock

# Create the Data object
data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features)

# Print the Data object to verify
print(data)

Data(x=[193, 5], edge_index=[2, 37056], edge_attr=[37056, 3])


In [ ]:
import pandas as pd

In [ ]:
import pandas as pd


file_path = '/content/drive/MyDrive/tickers.xlsx'


df_embed = pd.read_csv('/content/drive/MyDrive/full_df.csv')

In [ ]:
df= pd.read_csv('/content/drive/MyDrive/allfields_merged_df.csv')

In [ ]:
filtered_data = df[df['tic'].isin(common_stocks)]

In [ ]:
print(filtered_data)

              date        open        high         low       close    volume  \
1446    2016-06-20  101.598981  102.225120  101.557785  101.681366   5951000   
1447    2018-04-30  142.245131  145.060836  141.433238  144.620346  12954700   
1448    2018-07-03  136.777444  137.177282  135.864780  136.012543   1437700   
1449    2018-07-11  138.107338  138.820089  137.072987  137.872650   3203800   
1450    2018-07-18  138.941796  138.959171  137.168619  137.272919   3824000   
...            ...         ...         ...         ...         ...       ...   
382792  2024-03-27  128.138651  128.138651  125.175287  126.136642   2416400   
382793  2024-03-27  128.138651  128.138651  125.175287  126.136642   2416400   
382794  2024-03-27  128.138651  128.138651  125.175287  126.136642   2416400   
382795  2024-03-28  127.742225  128.812609  127.246678  128.198135   3204100   
382796  2024-03-28  127.742225  128.812609  127.246678  128.198135   3204100   

        dividends  stock splits   day  

In [ ]:
filtered_data.to_csv('/content/drive/MyDrive/filtered_data_final.csv', index=False)

In [ ]:
print(df)

              date        open        high         low       close   volume  \
0       2016-01-04  136.030029  138.060457  135.447208  137.759659  1461700   
1       2016-01-13  130.878716  131.724741  126.817845  127.146843  1671200   
2       2016-01-19  128.105634  132.946720  128.086829  131.527298  1843800   
3       2016-02-04  122.766367  126.874242  122.766367  126.686241  2151600   
4       2016-02-05  125.642832  127.005855  123.800390  125.069420  1567200   
...            ...         ...         ...         ...         ...      ...   
382792  2024-03-27  128.138651  128.138651  125.175287  126.136642  2416400   
382793  2024-03-27  128.138651  128.138651  125.175287  126.136642  2416400   
382794  2024-03-27  128.138651  128.138651  125.175287  126.136642  2416400   
382795  2024-03-28  127.742225  128.812609  127.246678  128.198135  3204100   
382796  2024-03-28  127.742225  128.812609  127.246678  128.198135  3204100   

        dividends  stock splits   day  tic  adj clo

In [ ]:
print(df_embed['symbol'])

actual_tickers = list(set(df_embed['symbol'].to_list()))
len(actual_tickers)

0       REG
1      KEYS
2       AEP
3       MRO
4      ALLE
       ... 
317    SEDG
318      CI
319      CE
320    FOXA
321    GOOG
Name: symbol, Length: 322, dtype: object


322

In [ ]:
# print(df)

In [ ]:
actual_tickers = list(set(df.tic.to_list()))
len(actual_tickers)

314

In [ ]:
def enumerate_companies(companies_dict):
    enumerated_companies = {}
    index = 0

    for company in companies_dict.keys():
        enumerated_companies[company] = index
        index += 1

    return enumerated_companies

enumerated_companies = enumerate_companies(relation_data)
enumerated_companies["AAPL"]

0

In [ ]:
# Max Drawdown
def calc_mdd(stock_prices):
    """Calculates the Maximal drawdown (MDD) of a given list.
    Returns MDD as absolute value and as percentage value
    """

    max_drawdown_abs = 0.0
    max_drawdown_pct = 0.0
    peak = stock_prices[0]  # Initialize peak as the first price
    for price in stock_prices:
        if price > peak:
            peak = price
        drawdown_abs = peak - price
        drawdown_pct = (peak - price) / peak
        if drawdown_abs > max_drawdown_abs:
            max_drawdown_abs = drawdown_abs
            max_drawdown_pct = drawdown_pct
    return max_drawdown_abs, max_drawdown_pct

In [ ]:
print(df.head())

         date        open        high         low       close    volume  \
0  2016-01-04   45.110001   45.110001   44.000000   44.919998    831700   
1  2016-01-04  135.361578  140.104897  133.783488  138.073349   1935300   
2  2016-01-04   13.366961   13.472861   13.335583   13.472861  42076058   
3  2016-01-04    9.488014    9.742231    9.217396    9.455212   6279800   
4  2016-01-04   60.795555   61.092220   59.866669   60.821110   4778100   

   dividends  stock splits  day   tic  adj close  close_pct_change  
0        0.0           0.0    0  SNPS        NaN               NaN  
1        0.0           0.0    0   AAP        NaN          2.073761  
2        0.0           0.0    0     T        NaN         -0.902422  
3        0.0           0.0    0   NRG        NaN         -0.298203  
4        0.0           0.0    0  ISRG        NaN          5.432549  


## Create Graph Data

In [ ]:

from torch_geometric.data import Data
import numpy as np
# Sample edge list and edge features (replace with your actual data)



# data for node features

node_features_list = []


for company in common_stocks:
    df_company = df[df['tic'] == company]


    company_stock_std = round(np.std(df_company.close, ddof=0),1)
    if company_stock_std == "nan":
        print(company, company_stock_std)

    company_volume_std = round(np.std(df_company.volume, ddof=0),1)

    #company_mdd = round(calc_mdd(df_company.high)[1], 1)

    comapny_dividens_sum = round(np.sum(df_company.dividends), 5)
    comapny_stocksplits_sum = round(np.sum(df_company["stock splits"]), 5)

    #percentage_increase = round(df_company.close.to_list()[-1] / df_company.close.to_list()[0], 3)

    company_upperband = round(np.std(df_company.upperband, ddof=0),2)
    company_lowerband = round(np.std(df_company.lowerband, ddof=0),2)
    company_middleband = round(np.std(df_company.middleband, ddof=0),2)
    company_moving_averages_list = round(np.std(df_company.moving_averages_list, ddof=0),2)
    company_cashflow = round(np.std(df_company.cashflow, ddof=0), 2)

    node_features_list.append([company_stock_std, company_volume_std])

print(node_features_list[0])

node_features = torch.tensor(node_features_list, dtype=torch.float)
# Number of nodes in the graph (assuming nodes are indexed from 0 to max_node_idx)
num_nodes = edge_index.max().item() + 1


data = Data(x=node_features, edge_index=edge_index, edge_attr=edge_features) # edge_features=edge_features

[29.4, 495275.4]


In [ ]:
np.random.seed(2)
# get the nodes
nodes = data.edge_index.t().numpy()
nodes = np.unique(list(nodes[:,0]) + list(nodes[:,1]))

# np.random.shuffle(nodes) # shuffle node order
print(len(nodes))

193


## Split dataset

In [ ]:
# get train test and val sizes: (70% - 15% - 15%)
train_size = int(len(nodes)*0.7)
test_size = int(len(nodes)*0.85) - train_size
val_size = len(nodes) - train_size - test_size

In [ ]:
# get train test and validation set of nodes
train_set = nodes[0:train_size]
test_set = nodes[train_size:train_size+test_size]
val_set = nodes[train_size+test_size:]


print(len(train_set),len(test_set),len(val_set))
print(len(train_set)+len(test_set)+len(val_set) == len(nodes))

print("train set\t",train_set[:10])
print("test set \t",test_set[:10])
print("val set  \t",val_set[:10])

135 29 29
True
train set	 [0 1 2 3 4 5 6 7 8 9]
test set 	 [135 136 137 138 139 140 141 142 143 144]
val set  	 [164 165 166 167 168 169 170 171 172 173]


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
# build test train val masks

train_mask = torch.zeros(len(nodes),dtype=torch.long, device=device)
for i in train_set:
    if i != 83:
        train_mask[i] = 1.

test_mask = torch.zeros(len(nodes),dtype=torch.long, device=device)
for i in test_set:
    test_mask[i] = 1.

val_mask = torch.zeros(len(nodes),dtype=torch.long, device=device)
for i in val_set:
    val_mask[i] = 1.

print("train mask \t",train_mask[0:15])
print("test mask  \t",test_mask[0:15])
print("val mask   \t",val_mask[0:15])

train mask 	 tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])
test mask  	 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
val mask   	 tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])


In [ ]:
# add masks
data.train_mask = train_mask
data.test_mask = test_mask
data.val_mask = val_mask

print("after\t\t",data)

after		 Data(x=[193, 2], edge_index=[2, 37056], edge_attr=[37056, 3], train_mask=[193], test_mask=[193], val_mask=[193])


In [ ]:
print(type(data))

<class 'torch_geometric.data.data.Data'>


In [ ]:
node_features_df = pd.DataFrame(data.x.numpy())
node_features_df.to_csv('/content/drive/MyDrive/node_features.csv', index=False)

# Convert edge index to a DataFrame and save to CSV
edge_index_df = pd.DataFrame(data.edge_index.numpy().T, columns=['source', 'target'])
edge_index_df.to_csv('/content/drive/MyDrive/edge_index.csv', index=False)

# If edge attributes are present, convert them to a DataFrame and save to CSV
if data.edge_attr is not None:
    edge_attr_df = pd.DataFrame(data.edge_attr.numpy())
    edge_attr_df.to_csv('/content/drive/MyDrive/edge_attributes.csv', index=False)

print("Data has been saved to CSV files.")

Data has been saved to CSV files.


In [ ]:
import pandas as pd
import torch
from torch_geometric.data import Data

In [ ]:
node_features_df = pd.read_csv('/content/drive/MyDrive/node_features.csv')
x = torch.tensor(node_features_df.values, dtype=torch.float)

# Load edge index from CSV
edge_index_df = pd.read_csv('/content/drive/MyDrive/edge_index.csv')
edge_index = torch.tensor(edge_index_df.values.T, dtype=torch.long)

# Load edge attributes from CSV (if exists)
try:
    edge_attr_df = pd.read_csv('/content/drive/MyDrive/edge_attributes.csv')
    edge_attr = torch.tensor(edge_attr_df.values, dtype=torch.float)
except FileNotFoundError:
    edge_attr = None

# Create the Data object
data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr)

In [ ]:
torch.set_num_threads(1)


In [ ]:
import torch
from torch_geometric.nn.models import Node2Vec

In [ ]:
print(torch.__version__)

In [ ]:
#!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-2.3.1+cpu121.html
#!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-2.3.1+cpu121.html
!pip install -q torch-cluster -f https://data.pyg.org/whl/torch-2.3.1+cpu121.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
!pip install pyg-lib -q

ERROR: Could not find a version that satisfies the requirement pyg-lib (from versions: none)
ERROR: No matching distribution found for pyg-lib


In [ ]:
# !condacolab KERNEL RESTART
# print("Restarting of kernel...")
# get_ipython().kernel.do_shutdown(True)

/bin/bash: line 1: condacolab: command not found
Restarting of kernel...


{'status': 'ok', 'restart': True}

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("Device:", device)

model = Node2Vec(data.edge_index, embedding_dim=128, walk_length=20,
             context_size=10, walks_per_node=10,
             num_negative_samples=1, p=1, q=1, sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=4)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)

torch.set_num_threads(1)

Device: cpu


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
torch.set_num_threads(1)

## Train Node2Vec

In [ ]:
def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


@torch.no_grad()
def test():
    model.eval()
    z = model()
    acc = model.test(z[data.train_mask], data.y[data.train_mask],
                     z[data.test_mask], data.y[data.test_mask],
                     max_iter=10)
    return acc


for epoch in range(1, 201):
    loss = train()
    #acc = test()
    if epoch % 10 == 0:
        print(f'Epoch: {epoch:02d}, Loss: {loss:.4f}')


Epoch: 10, Loss: 7.1858
Epoch: 20, Loss: 5.2104
Epoch: 30, Loss: 3.9636
Epoch: 40, Loss: 3.2485
Epoch: 50, Loss: 2.7811
Epoch: 60, Loss: 2.4585
Epoch: 70, Loss: 2.2584
Epoch: 80, Loss: 2.1085
Epoch: 90, Loss: 2.0003
Epoch: 100, Loss: 1.9140
Epoch: 110, Loss: 1.8787
Epoch: 120, Loss: 1.7988
Epoch: 130, Loss: 1.7813
Epoch: 140, Loss: 1.7608
Epoch: 150, Loss: 1.7518
Epoch: 160, Loss: 1.7326
Epoch: 170, Loss: 1.7128
Epoch: 180, Loss: 1.7072
Epoch: 190, Loss: 1.6804
Epoch: 200, Loss: 1.6810


## Detach embeddings and  store all to csv

In [ ]:
z = model()

# from tensor to numpy
emb = z.detach().cpu().numpy()

In [ ]:
csv_file_path = "/content/drive/MyDrive/node2vec_embeedings.csv"
csv_file_path

'/content/drive/MyDrive/node2vec_embeedings.csv'

In [ ]:
import csv
# Write the data to the CSV file
with open(csv_file_path, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    for row in emb:
        csv_writer.writerow(row)

In [ ]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/node2vec_embeedings.csv')


In [ ]:
print(df)

     -0.8028928  0.6364976  0.089480005  0.52772045  -0.35956797  -0.77462703  \
0     -0.091657  -0.022072    -0.428367   -0.132408     0.199133    -0.044436   
1      0.054191  -0.165534    -0.077147   -0.096555    -0.160630     0.108926   
2      0.051981  -0.028293     0.086656    0.013554     0.039774    -0.010397   
3      0.916243  -0.313245     0.508415    0.728061    -0.430762    -0.852728   
4      0.027382  -0.036152     0.289892   -0.244340    -0.084346    -0.059694   
..          ...        ...          ...         ...          ...          ...   
187    0.347121   0.614322     0.104573    1.390100     0.268391     0.214828   
188    0.012300   0.051958    -0.025094   -0.209117     0.002489    -0.111686   
189   -0.442247   0.293643     0.455382    0.115346    -0.311267     0.199858   
190   -0.273133   0.562157    -0.098264   -0.670370    -0.011363    -0.483353   
191    0.171485   0.217481    -0.001379    0.171704    -0.005342     0.078376   

     0.16475296  0.8548588 